### Импорт необходимых библиотек

In [46]:
import pandas as pd # Для чтения датафрейма
from sklearn.preprocessing import LabelEncoder # для кодирования переменных
from sklearn.model_selection import train_test_split # для разделения на трейн и тест
from sklearn.utils import resample # Для апсемплинга
from sklearn.ensemble import AdaBoostClassifier # по заданию
from sklearn.ensemble import GradientBoostingClassifier  # по заданию
from sklearn.metrics import classification_report # для вывода результата работы модели

### Чтение данных

In [2]:
data = pd.read_csv('BankChurners.csv')

In [49]:
### Удалим признаки, которые были отмечены создателем датасета на каггле
data = data.drop(columns=data.columns[-2:])

In [10]:
### также для моделирования не нужен номер клиента
data = data.drop(columns='CLIENTNUM')

### Кодирование переменных
Не имеет смысла еще как-то предобрабатывать данные для модели бустинга, так как в данных нет пропусков, а все численные значения обработают деревья в алгоритмах бустинга

In [52]:
data.isna().any()

Attrition_Flag              False
Customer_Age                False
Gender                      False
Dependent_count             False
Education_Level             False
Marital_Status              False
Income_Category             False
Card_Category               False
Months_on_book              False
Total_Relationship_Count    False
Months_Inactive_12_mon      False
Contacts_Count_12_mon       False
Credit_Limit                False
Total_Revolving_Bal         False
Avg_Open_To_Buy             False
Total_Amt_Chng_Q4_Q1        False
Total_Trans_Amt             False
Total_Trans_Ct              False
dtype: bool

In [14]:
le = LabelEncoder()

In [22]:
columns_to_transform = data.columns[data.dtypes=='object'].tolist()

In [24]:
for i in columns_to_transform:
    data[i] = le.fit_transform(data[i])

### Деление на трейн и тест

In [27]:
data_train, data_test = train_test_split(data, test_size=0.25)

### Апсемплинг

In [32]:
positive_samples = data_train[data_train.Attrition_Flag==1]
negative_samples = data_train[data_train.Attrition_Flag==0]

In [33]:
positive_samples.shape, negative_samples.shape

((6384, 20), (1211, 20))

In [35]:
negative_samples = resample(negative_samples, n_samples=len(positive_samples))

In [39]:
data_train = pd.concat((positive_samples, negative_samples)).sample(frac=1)

### Финальная обработка выборок

In [41]:
X_train = data_train.drop(columns='Attrition_Flag')
y_train = data_train.Attrition_Flag

X_test = data_test.drop(columns='Attrition_Flag')
y_test = data_test.Attrition_Flag

In [44]:
adaboost = AdaBoostClassifier()
gradient = GradientBoostingClassifier()

### Построение предсказаний

In [45]:
adaboost.fit(X_train, y_train)
gradient.fit(X_train, y_train)

GradientBoostingClassifier()

In [47]:
print(classification_report(y_test, adaboost.predict(X_test)))

              precision    recall  f1-score   support

           0       0.74      0.91      0.82       416
           1       0.98      0.94      0.96      2116

    accuracy                           0.93      2532
   macro avg       0.86      0.92      0.89      2532
weighted avg       0.94      0.93      0.94      2532



In [48]:
print(classification_report(y_test, gradient.predict(X_test)))

              precision    recall  f1-score   support

           0       0.79      0.95      0.86       416
           1       0.99      0.95      0.97      2116

    accuracy                           0.95      2532
   macro avg       0.89      0.95      0.92      2532
weighted avg       0.96      0.95      0.95      2532



### Градиентный бустинг показал себя лучше